In [1]:
import os
from options.test_options import TestOptions
from data import CreateDataLoader
from models import create_model
from util.visualizer import Visualizer
from util import html
import torch
from models.cycle_gan_model import CycleGANModel
import matplotlib.pyplot as plt
from torchsummary import summary

In [2]:
opt = TestOptions().parse()
opt.dataroot = 'datasets/horse2zebra/testA'
opt.checkpoints_dir = './checkpoints/'
opt.name = 'horse2zebra_pretrained'
opt.no_dropout = True
opt.model = 'test'
opt.dataset_mode = 'single'
opt.loadSize = 256
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.display_id = -1  # no visdom display
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
model = create_model(opt)

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: None
dataset_mode: unaligned
display_id: 1
display_port: 8097
display_server: http://localhost
display_winsize: 256
fineSize: 256
gpu_ids: [0]
how_many: 50
init_type: normal
input_nc: 3
isTrain: False
loadSize: 286
max_dataset_size: inf
model: cycle_gan
nThreads: 4
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
no_dropout: False
no_flip: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
suffix: 
verbose: False
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
-------------- End ----------------
CustomDatasetDataLoader
dataset [SingleImageDataset] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 11.378 M
-----------------------------------------------
model [TestModel] was created


In [3]:
dick = torch.load('/home/toby/Documents/pytorch-CycleGAN-and-pix2pix/checkpoints/yosemite_cyclegan/latest_net_G_A.pth')

for key, value in dick.items():
    print(key)

down.1.weight
down.1.bias
down.4.weight
down.4.bias
down.7.weight
down.7.bias
down.10.conv_block.1.weight
down.10.conv_block.1.bias
down.10.conv_block.5.weight
down.10.conv_block.5.bias
down.11.conv_block.1.weight
down.11.conv_block.1.bias
down.11.conv_block.5.weight
down.11.conv_block.5.bias
down.12.conv_block.1.weight
down.12.conv_block.1.bias
down.12.conv_block.5.weight
down.12.conv_block.5.bias
down.13.conv_block.1.weight
down.13.conv_block.1.bias
down.13.conv_block.5.weight
down.13.conv_block.5.bias
down.14.conv_block.1.weight
down.14.conv_block.1.bias
down.14.conv_block.5.weight
down.14.conv_block.5.bias
down.15.conv_block.1.weight
down.15.conv_block.1.bias
down.15.conv_block.5.weight
down.15.conv_block.5.bias
down.16.conv_block.1.weight
down.16.conv_block.1.bias
down.16.conv_block.5.weight
down.16.conv_block.5.bias
down.17.conv_block.1.weight
down.17.conv_block.1.bias
down.17.conv_block.5.weight
down.17.conv_block.5.bias
down.18.conv_block.1.weight
down.18.conv_block.1.bias
down

In [4]:
from collections import OrderedDict
new_dict = OrderedDict()
for key, value in dick.items():
    names = key.split('.')
    if len(names) == 3 and int(names[1]) < 10:
        new_key = 'module.' + key.replace('model', 'down')
    else:
        new_key = 'module.up'
        new_key += '.' + str(int(names[1]) - 10)
        for name in names[2:]:
            new_key += '.' + name
    new_dict[new_key] = value
    #print(new_key)
#torch.save(new_dict, './checkpoints/horse2zebra_pretrained/updated_net_G.pth')

ValueError: invalid literal for int() with base 10: 'down'

In [4]:
for k in model.netG.state_dict().keys():
    print(k)
model.netG.module.load_state_dict(dick)

module.down.1.weight
module.down.1.bias
module.down.4.weight
module.down.4.bias
module.down.7.weight
module.down.7.bias
module.down.10.conv_block.1.weight
module.down.10.conv_block.1.bias
module.down.10.conv_block.5.weight
module.down.10.conv_block.5.bias
module.down.11.conv_block.1.weight
module.down.11.conv_block.1.bias
module.down.11.conv_block.5.weight
module.down.11.conv_block.5.bias
module.down.12.conv_block.1.weight
module.down.12.conv_block.1.bias
module.down.12.conv_block.5.weight
module.down.12.conv_block.5.bias
module.down.13.conv_block.1.weight
module.down.13.conv_block.1.bias
module.down.13.conv_block.5.weight
module.down.13.conv_block.5.bias
module.down.14.conv_block.1.weight
module.down.14.conv_block.1.bias
module.down.14.conv_block.5.weight
module.down.14.conv_block.5.bias
module.down.15.conv_block.1.weight
module.down.15.conv_block.1.bias
module.down.15.conv_block.5.weight
module.down.15.conv_block.5.bias
module.down.16.conv_block.1.weight
module.down.16.conv_block.1.b

In [5]:
data_iter = iter(data_loader)
data = next(data_iter)
result = model.netG(data['A'],cache=True)

In [6]:
model.netG.module.intermediate.shape
data['A'].shape

torch.Size([1, 3, 256, 256])

In [7]:
summary(model.netG, (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1          [-1, 3, 262, 262]               0
            Conv2d-2         [-1, 64, 256, 256]            9472
    InstanceNorm2d-3         [-1, 64, 256, 256]               0
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5        [-1, 128, 128, 128]           73856
    InstanceNorm2d-6        [-1, 128, 128, 128]               0
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8          [-1, 256, 64, 64]          295168
    InstanceNorm2d-9          [-1, 256, 64, 64]               0
             ReLU-10          [-1, 256, 64, 64]               0
  ReflectionPad2d-11          [-1, 256, 66, 66]               0
           Conv2d-12          [-1, 256, 64, 64]          590080
   InstanceNorm2d-13          [-1, 256, 64, 64]               0
             ReLU-14          [-1, 256,